In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# Burgers 방정식의 PINN 모델 정의
class BurgersPINN(nn.Module):
    def __init__(self):
        super(BurgersPINN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 50), nn.ReLU(),
            nn.Linear(50, 1)
        )

    def forward(self, x, t):
        input_data = torch.cat((x, t), dim=1)
        return self.model(input_data)

In [ ]:
# 초기 조건 함수
def initial_condition(x):
    return -np.sin(np.pi * x)

# 경계 조건 함수
def boundary_condition(x, t):
    return np.zeros_like(x)

# Burgers 방정식 정의
def burgers_equation(u, x, t):
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True, allow_unused=True)[0]
    u_t = torch.autograd.grad(u, t, torch.ones_like(t), create_graph=True, allow_unused=True)[0]
    #u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True, allow_unused=True)[0]

    return u_t + u * u_x

In [ ]:
# PINN 학습 함수
def train_PINN(model, x_initial, t_initial, x_boundary, t_boundary, x_collocation, t_collocation, epochs, learning_rate):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()

        x_initial_tensor, t_initial_tensor, x_boundary_tensor, t_boundary_tensor = (
            torch.tensor(x, dtype=torch.float32, requires_grad=True) for x in
            [x_initial, t_initial, x_boundary, t_boundary]
        )

        u_initial_pred, u_boundary_pred = (
            model(x, t) for x, t in
            [(x_initial_tensor, t_initial_tensor), (x_boundary_tensor, t_boundary_tensor)]
        )

        u_initial_exact, u_boundary_exact = (
            torch.tensor(u, dtype=torch.float32) for u in [initial_condition(x_initial), boundary_condition(x_boundary, t_boundary)]
        )

        loss = loss_fn(u_initial_pred, u_initial_exact) + loss_fn(u_boundary_pred, u_boundary_exact)

        # 랜덤한 Collocation points 생성
        idx = np.random.choice(len(x_collocation), size=100, replace=False)
        x_collocation_batch = x_collocation[idx]
        t_collocation_batch = t_collocation[idx]

        x_collocation_tensor, t_collocation_tensor = (
            torch.tensor(x_collocation_batch, dtype=torch.float32, requires_grad=True) for x_collocation_batch in
            [x_collocation_batch, t_collocation_batch]
        )

        u_collocation_pred = model(x_collocation_tensor, t_collocation_tensor)

        equation_residual = burgers_equation(u_collocation_pred, x_collocation_tensor, t_collocation_tensor)
        loss += loss_fn(equation_residual, torch.zeros_like(equation_residual))

        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item()}")

    print("Training completed.")

In [ ]:
# 학습 데이터 생성
x_initial = np.linspace(0, 1, 100).reshape(-1, 1)
t_initial = np.zeros_like(x_initial)
x_boundary = np.linspace(0, 1, 100).reshape(-1, 1)
t_boundary = np.linspace(0, 1, 100).reshape(-1, 1)
x_collocation = np.random.uniform(0, 1, size=(1000, 1))
t_collocation = np.random.uniform(0, 1, size=(1000, 1))

# 모델 생성 및 학습
model = BurgersPINN()
train_PINN(model, x_initial, t_initial, x_boundary, t_boundary, x_collocation, t_collocation, epochs=10000, learning_rate=0.001)


In [ ]:
# 결과 시각화
X_visualize, T_visualize = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
X_visualize_tensor = torch.tensor(X_visualize, dtype=torch.float32)
T_visualize_tensor = torch.tensor(T_visualize, dtype=torch.float32)


# 데이터를 1D로 변환
X_visualize_flat = X_visualize_tensor.view(-1)
T_visualize_flat = T_visualize_tensor.view(-1)

u_visualize = model(X_visualize_flat.view(-1, 1), T_visualize_flat.view(-1, 1)).detach().numpy()

plt.figure(figsize=(10, 6))
plt.pcolormesh(T_visualize, X_visualize, u_visualize.reshape(100, 100), cmap='viridis')
plt.scatter(t_collocation, x_collocation, c="red", marker='.')
plt.colorbar(label='u(x,t)')
plt.xlabel('t')
plt.ylabel('x')
plt.title('Burgers Equation Solution')
plt.show()